In [ ]:
CHALLENGE - 3 - topic analysis

In [8]:
#IMPORTING

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint
import string
import os
import re


import matplotlib.pyplot as plt
import seaborn as sns


from wordcloud import STOPWORDS,WordCloud
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


import gensim
from gensim.models import CoherenceModel
from gensim import corpora



file_name = 'tweet_data.csv'
df = pd.read_csv(file_name)


In [9]:
df.head(2)

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Fri Jun 18 17:55:49 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","🚨Africa is ""in the midst of a full-blown third...",0.166667,0.188889,en,548,612.0,ketuesriche,551,351,NaN,NaN,NaN,Mass
1,Fri Jun 18 17:55:59 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Dr Moeti is head of WHO in Africa, and one of ...",0.133333,0.455556,en,195,92.0,Grid1949,66,92,NaN,NaN,NaN,"Edinburgh, Scotland"


In [10]:
df.shape

(6532, 15)

In [11]:
df.columns

Index(['created_at', 'source', 'original_text', 'polarity', 'subjectivity',
       'lang', 'favorite_count', 'retweet_count', 'original_author',
       'followers_count', 'friends_count', 'possibly_sensitive', 'hashtags',
       'user_mentions', 'place'],
      dtype='object')

In [12]:
count_texts = df['original_text'].isnull().sum()
print("Data Frame before removing null rows ", df.shape)
print("Number of null orignal_text ", count_texts)

Data Frame before removing null rows  (6532, 15)
Number of null orignal_text  2812


In [13]:
df = df[~df['original_text'].isnull()]
count_texts = df['original_text'].isnull().sum()
print("Data Frame after removing null rows ", df.shape)
print("Number of null orignal_text ", count_texts)

Data Frame after removing null rows  (3720, 15)
Number of null orignal_text  0


In [14]:
import re
def clean_text(text):
    hash_tag_removed = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
    removed_links = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', hash_tag_removed, flags=re.MULTILINE)
    result = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', removed_links)
    return result

df['original_text'] = df['original_text'].apply(clean_text)

In [15]:
cleanTweet = pd.DataFrame()
cleanTweet['clean_text'] = df['original_text']
cleanTweet['polarity'] = df['polarity']

cleanTweet.head()

,clean_text,polarity
0,"🚨Africa is ""in the midst of a full-blown third...",0.166667
1,"Dr Moeti is head of WHO in Africa, and one of ...",0.133333
2,Thank you for creating this amazing campaign ...,0.316667
3,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111
4,I think it’s important that we don’t sell COVA...,0.280000


In [16]:
cleanTweet.columns

Index(['clean_text', 'polarity'], dtype='object')

In [17]:
cleanTweet.dropna()

,clean_text,polarity
0,"🚨Africa is ""in the midst of a full-blown third...",0.166667
1,"Dr Moeti is head of WHO in Africa, and one of ...",0.133333
2,Thank you for creating this amazing campaign ...,0.316667
3,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111
4,I think it’s important that we don’t sell COVA...,0.280000
...,...,...
6521,Australia is sending vaccines.\nAustralia is s...,0.100000
6522,The Truth Behind COVID-19 Vaccines (6) —— Bell...,-0.386111
6524,Covid19 vaccines reach the remotest places of ...,-0.050000
6528,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111


In [19]:
def process_data():
    cleanTweet['clean_text'] = cleanTweet['clean_text'].apply(lambda x: x.lower())
    cleanTweet['clean_text'] = cleanTweet['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    sentence_list = [tweet for tweet in cleanTweet['clean_text']]
    word_list = [sentence.split() for sentence in sentence_list]
    
    word_to_id = corpora.Dictionary(word_list)
    corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]
    
    return word_list, word_to_id, corpus_1

In [20]:
word_list, id2word, corpus = process_data()
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 3), (21, 1), (22, 2), (23, 1), (24, 4), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]


In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [22]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('the', 0.059252042),
   ('of', 0.04634107),
   ('in', 0.025933292),
   ('and', 0.020663034),
   ('to', 0.018838506),
   ('vaccines', 0.016757308),
   ('by', 0.016396984),
   ('have', 0.015626783),
   ('amp', 0.013902195),
   ('on', 0.012009597)]),
 (1,
  [('in', 0.044179697),
   ('the', 0.03923091),
   ('india', 0.037722167),
   ('of', 0.03573105),
   ('a', 0.027385112),
   ('and', 0.021746434),
   ('wave', 0.02091004),
   ('africa', 0.020825425),
   ('amp', 0.020103801),
   ('third', 0.020095455)]),
 (2,
  [('to', 0.058688473),
   ('vaccines', 0.033360817),
   ('need', 0.032535393),
   ('we', 0.029929623),
   ('the', 0.026091753),
   ('are', 0.023548318),
   ('and', 0.023392605),
   ('you', 0.021670066),
   ('from', 0.016076073),
   ('with', 0.015258031)]),
 (3,
  [('to', 0.069139645),
   ('the', 0.036816932),
   ('is', 0.028010888),
   ('of', 0.02688309),
   ('and', 0.026510037),
   ('africa', 0.021838443),
   ('vaccines', 0.019329678),
   ('australia', 0.01771431),
   ('in'

In [23]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -6.079594595910208

 Ldamodel Coherence Score/Accuracy on Tweets:  0.41991022045536186


In [24]:
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.064693  0.113121       1        1  26.433145
0     -0.127024 -0.013647       2        1  25.106812
3      0.003640  0.200493       3        1  19.778449
2      0.261863 -0.152088       4        1  15.150264
1     -0.203171 -0.147879       5        1  13.531329, topic_info=       Term         Freq        Total Category  logprob  loglift
244   india   681.000000   681.000000  Default  30.0000  30.0000
205    need   658.000000   658.000000  Default  29.0000  29.0000
54       to  5001.000000  5001.000000  Default  28.0000  28.0000
15       in  2925.000000  2925.000000  Default  27.0000  27.0000
59       we   850.000000   850.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
16       is   250.624982  1903.480669   Topic5  -4.2765  -0.0273
54       to   317.691211  5001.302686   Topic5  -4.0394  -0